In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

In [2]:
#Getting fine food reviews data from kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"akshat4112","key":"1417d9530d822ed9de2833480162ad84"}'}

In [4]:
!pip install -q kaggle

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [6]:
!kaggle datasets download -d snap/amazon-fine-food-reviews
!kaggle datasets download -d leadbest/googlenewsvectorsnegative300

 95% 231M/242M [00:01<00:00, 162MB/s]
100% 242M/242M [00:01<00:00, 152MB/s]
100% 3.17G/3.17G [00:38<00:00, 78.6MB/s]
100% 3.17G/3.17G [00:38<00:00, 89.1MB/s]


In [7]:
!unzip \*.zip

Archive:  amazon-fine-food-reviews.zip
  inflating: Reviews.csv             
  inflating: database.sqlite         
  inflating: hashes.txt              

Archive:  googlenewsvectorsnegative300.zip
  inflating: GoogleNews-vectors-negative300.bin  
  inflating: GoogleNews-vectors-negative300.bin.gz  

2 archives were successfully processed.


In [8]:
!ls

amazon-fine-food-reviews.zip	       hashes.txt
database.sqlite			       kaggle.json
GoogleNews-vectors-negative300.bin     Reviews.csv
GoogleNews-vectors-negative300.bin.gz  sample_data
googlenewsvectorsnegative300.zip


In [9]:
#Getting word2vec data from kaggle

In [10]:
con = sqlite3.connect('database.sqlite')

In [11]:
filter_data = pd.read_sql_query("""SELECT * FROM Reviews WHERE Score !=3""", con)

In [12]:
def partition(x):
    if x<3:
        return 'negative'
    return 'positive'

In [13]:
actualScore = filter_data['Score']
PositiveNegative = actualScore.map(partition)
filter_data['Score'] = PositiveNegative

In [14]:
filter_data.shape
filter_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [15]:
display = pd.read_sql_query("""SELECT * FROM Reviews WHERE Score !=3 AND UserId = "AR5J8UI46CURR" ORDER BY ProductID""", con)

In [16]:
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [17]:
sorted_data = filter_data.sort_values('ProductId', axis=0, ascending=True)

In [18]:
final = sorted_data.drop_duplicates(subset= {"UserId", "ProfileName", "Time", "Text"}, keep='first', inplace=False)
final.shape

(364173, 10)

In [19]:
(final['Id'].size*1.0)/(filter_data['Id'].size*1.0)*100

69.25890143662969

In [20]:
display = pd.read_sql_query("""SELECT * FROM Reviews WHERE Score!=3 AND Id=44737 OR Id=64422 ORDER BY ProductID""", con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [21]:
final = final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [22]:
final.shape

(364171, 10)

In [23]:
final['Score'].value_counts()

positive    307061
negative     57110
Name: Score, dtype: int64

In [24]:
#Basic Text Processing 

In [25]:
#Given these 8 features, predict sentiment polarity i.e. score
#using BOW
#using tf-idf
#using word2vec
#using average word2vec

#remove stop words
#convert all the lower case
#stemming 
#lemmaization

In [26]:
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer #this one is better
from nltk.stem.wordnet import WordNetLemmatizer

In [27]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [28]:
stop = set(stopwords.words('english'))
sno = nltk.stem.SnowballStemmer('english')

In [29]:
def cleanhtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext

In [30]:
def cleanpunch(sentence):
    cleaned = re.sub(r'[?|\'|"|#]', r'',sentence)
    cleaned = re.sub(r'[.|,|)|\|/]', r'',sentence)
    return cleaned

In [31]:
print(stop)

{'of', "didn't", "isn't", 'have', 'against', 'both', 'or', 'to', 'this', 'too', 'yourself', 'my', 'before', 'some', 'ourselves', 'doing', 'more', 'an', "mustn't", 'these', 'ain', 'won', 'ours', 'out', 'i', 'if', 'myself', 'being', 'is', 'y', 'while', "should've", "hadn't", "needn't", 'themselves', 'here', 'mightn', 'does', 'which', 'off', 'now', 'few', "wasn't", 'through', 'over', 'having', 'had', 've', 't', "shouldn't", "you've", 'for', 'from', 'did', 'your', 'them', 'any', 'down', 'will', 'yourselves', 'by', 'those', 'because', 'hers', "hasn't", 'am', 'me', 'll', 'are', 'what', 'about', 'between', 'and', 'until', 'all', 'under', 'very', 'wouldn', 'its', "aren't", 'be', 'where', 'most', 'below', 'o', 'him', 'hasn', 'didn', 'haven', 'once', 'been', 'other', 'no', 'should', "doesn't", 'her', 's', 'ma', 'weren', 'shan', "won't", 'again', "haven't", 'only', 'our', 'on', 'so', 'whom', 'don', 'after', 'not', 'it', 'do', 'theirs', 'in', "couldn't", 'that', 'nor', 'who', 'when', "that'll", "w

In [32]:
#BOW
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
final_counts = count_vect.fit_transform(final['Text'].values)

In [33]:
type(final_counts)

scipy.sparse.csr.csr_matrix

In [34]:
final_counts.get_shape()

(364171, 115281)

In [35]:
#Text Preprocessing

In [36]:
#find sentence with HTML tags

i = 0 ;
for sent in final['Text'].values:
    if(len(re.findall('<.*?>', sent))):
        print(i)
        print(sent)
        break;
    i+=1

6
I set aside at least an hour each day to read to my son (3 y/o). At this point, I consider myself a connoisseur of children's books and this is one of the best. Santa Clause put this under the tree. Since then, we've read it perpetually and he loves it.<br /><br />First, this book taught him the months of the year.<br /><br />Second, it's a pleasure to read. Well suited to 1.5 y/o old to 4+.<br /><br />Very few children's books are worth owning. Most should be borrowed from the library. This book, however, deserves a permanent spot on your shelf. Sendak's best.


In [37]:
print(sno.stem("tasty"))

tasti


In [38]:
i = 0
str1 = ' '
final_string = []
all_positive_words = []
all_negavtove_words = []
s = ''

for sent in final['Text'].values:
    filtered_sentence = []
    sent = cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunch(w).split():
            if ((cleaned_words.isalpha()) & (len(cleaned_words)> 2)):
                if (cleaned_words.lower() not in stop):
                    s = (sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if (final['Score'].values)[i] == 'positive':
                        all_positive_words.append(s)
                    if (final['Score'].values)[i] == 'negative':
                        all_negavtove_words.append(s)
                else:
                    continue
            else:
                continue
    str1 = b" ".join(filtered_sentence)
    final_string.append(str1)
    i+=1
#     print(str1)

In [39]:
final["CleanedText"] = final_string

In [40]:
final.head(3)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,b'witti littl book make son laugh loud recit c...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",b'grew read sendak book watch realli rosi movi...
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,b'fun way children learn month learn poem thro...


In [41]:
conn = sqlite3.connect('final.sqlite')
c = conn.cursor()
conn.text_factory = str
final.to_sql('Reviews', conn, schema=None, if_exists='replace')

In [42]:
#Bi Grams and n-Grams

In [43]:
freq_dist_positive = nltk.FreqDist(all_positive_words)
freq_dist_negative = nltk.FreqDist(all_negavtove_words)
print("Most Common Positive Words: ", freq_dist_positive.most_common(20))
print("Most Common Negative Words: ", freq_dist_negative.most_common(20))

Most Common Positive Words:  [(b'like', 137225), (b'tast', 126067), (b'good', 107434), (b'love', 106255), (b'flavor', 106249), (b'use', 102993), (b'great', 98171), (b'one', 94371), (b'product', 86398), (b'tri', 85280), (b'tea', 80592), (b'coffe', 75707), (b'make', 74571), (b'get', 71719), (b'food', 62442), (b'would', 55377), (b'time', 53596), (b'buy', 53458), (b'realli', 52368), (b'eat', 51157)]
Most Common Negative Words:  [(b'tast', 33853), (b'like', 31917), (b'product', 27327), (b'one', 20105), (b'flavor', 18746), (b'would', 17923), (b'tri', 17626), (b'use', 15139), (b'good', 14566), (b'coffe', 14179), (b'get', 13725), (b'buy', 13562), (b'order', 12731), (b'food', 12282), (b'tea', 11252), (b'even', 10912), (b'box', 10509), (b'make', 9793), (b'time', 9577), (b'bag', 9456)]


In [44]:
count_vect = CountVectorizer(ngram_range=(1,2))
final_bigram_count = count_vect.fit_transform(final['Text'].values)

In [45]:
final_bigram_count.get_shape()

(364171, 2910192)

In [46]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

In [47]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2)) 
final_tf_idf=  tf_idf_vect.fit_transform(final['Text'].values)

In [48]:
final_tf_idf.get_shape()

(364171, 2910192)

In [49]:
features = tf_idf_vect.get_feature_names()
len(features)

2910192

In [50]:
features[100000:100010]

['ales until',
 'ales ve',
 'ales would',
 'ales you',
 'alessandra',
 'alessandra ambrosia',
 'alessi',
 'alessi added',
 'alessi also',
 'alessi and']

In [51]:
print(final_tf_idf[3,:].toarray()[0])

[0. 0. 0. ... 0. 0. 0.]


In [52]:
def top_tfidf_feats(row, features, top_n=25):
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df_columns = ['feature', 'tfidf']
    return df
    

In [53]:
top_tfidf = top_tfidf_feats(final_tf_idf[1,:].toarray()[0], features,25)

In [54]:
top_tfidf

,0,1
0,sendak books,0.173437
1,rosie movie,0.173437
2,paperbacks seem,0.173437
3,cover version,0.173437
4,these sendak,0.173437
5,the paperbacks,0.173437
6,pages open,0.173437
7,really rosie,0.168074
8,incorporates them,0.168074
9,paperbacks,0.168074


Word2vec

In [55]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

In [57]:
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True, unicode_errors='ignore')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [58]:
model.wv['computer']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


array([ 1.07421875e-01, -2.01171875e-01,  1.23046875e-01,  2.11914062e-01,
       -9.13085938e-02,  2.16796875e-01, -1.31835938e-01,  8.30078125e-02,
        2.02148438e-01,  4.78515625e-02,  3.66210938e-02, -2.45361328e-02,
        2.39257812e-02, -1.60156250e-01, -2.61230469e-02,  9.71679688e-02,
       -6.34765625e-02,  1.84570312e-01,  1.70898438e-01, -1.63085938e-01,
       -1.09375000e-01,  1.49414062e-01, -4.65393066e-04,  9.61914062e-02,
        1.68945312e-01,  2.60925293e-03,  8.93554688e-02,  6.49414062e-02,
        3.56445312e-02, -6.93359375e-02, -1.46484375e-01, -1.21093750e-01,
       -2.27539062e-01,  2.45361328e-02, -1.24511719e-01, -3.18359375e-01,
       -2.20703125e-01,  1.30859375e-01,  3.66210938e-02, -3.63769531e-02,
       -1.13281250e-01,  1.95312500e-01,  9.76562500e-02,  1.26953125e-01,
        6.59179688e-02,  6.93359375e-02,  1.02539062e-02,  1.75781250e-01,
       -1.68945312e-01,  1.21307373e-03, -2.98828125e-01, -1.15234375e-01,
        5.66406250e-02, -

In [59]:
model.wv.similarity('man', 'woman')

In [60]:
model.wv.most_similar('woman')

In [60]:
model.wv.most_similar('tasty')

In [61]:
model.wv.similarity('tasty', 'tast')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.44035053

In [62]:
#Train your own word2vec model using your own text

In [71]:
import gensim

i = 0
list_of_sent = []

for sent in final['Text'].values:
    filtered_sentence = []
    sent = cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunch(w).split():
          if (cleaned_words.isalpha()):
            filtered_sentence.append(cleaned_words.lower())
          else:
            continue
    list_of_sent.append(filtered_sentence)
  

In [72]:
print(final['Text'].values[0])
print("*"*20)
print(list_of_sent[0])

this witty little book makes my son laugh at loud. i recite it in the car as we're driving along and he always can sing the refrain. he's learned about whales, India, drooping roses:  i love all the new words this book  introduces and the silliness of it all.  this is a classic book i am  willing to bet my son will STILL be able to recite from memory when he is  in college
********************
['this', 'witty', 'little', 'book', 'makes', 'my', 'son', 'laugh', 'at', 'loud', 'i', 'recite', 'it', 'in', 'the', 'car', 'as', 'driving', 'along', 'and', 'he', 'always', 'can', 'sing', 'the', 'refrain', 'learned', 'about', 'whales', 'india', 'drooping', 'i', 'love', 'all', 'the', 'new', 'words', 'this', 'book', 'introduces', 'and', 'the', 'silliness', 'of', 'it', 'all', 'this', 'is', 'a', 'classic', 'book', 'i', 'am', 'willing', 'to', 'bet', 'my', 'son', 'will', 'still', 'be', 'able', 'to', 'recite', 'from', 'memory', 'when', 'he', 'is', 'in', 'college']


In [73]:
w2v_model = gensim.models.Word2Vec(list_of_sent, min_count =5, size=50, workers=4)

In [74]:
words = list(w2v_model.wv.vocab)
print(len(words))

33479


In [75]:
w2v_model.wv.most_similar('tasty')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('tastey', 0.896468460559845),
 ('yummy', 0.868288516998291),
 ('satisfying', 0.8386591672897339),
 ('delicious', 0.8249577879905701),
 ('filling', 0.8237217664718628),
 ('flavorful', 0.7959707975387573),
 ('tasteful', 0.770639181137085),
 ('nutritious', 0.758531928062439),
 ('hardy', 0.7558744549751282),
 ('versatile', 0.7558155655860901)]

In [76]:
w2v_model.wv.most_similar('like')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('prefer', 0.6917452812194824),
 ('resemble', 0.6430633664131165),
 ('dislike', 0.6306471824645996),
 ('alright', 0.6101357340812683),
 ('enjoy', 0.6019206047058105),
 ('fake', 0.5992211103439331),
 ('mean', 0.5931571125984192),
 ('hate', 0.5908896923065186),
 ('gross', 0.5886486768722534),
 ('love', 0.577628493309021)]

In [77]:
count_vect_feat = count_vect.get_feature_names()
count_vect_feat.index('like')
print(count_vect_feat[64055])

activity great


In [83]:
#Average Word2Vec

In [81]:
sent_vectors = []
for sent in list_of_sent:
  sent_vec = np.zeros(50)
  cnt_words = 0
  for word in sent:
    try:
      vec = w2v_model.wv[word]
      sent_vec+=vec
      cnt_words+=1
    except:
      pass
  sent_vec /=cnt_words
  sent_vectors .append(sent_vec)
print(len(sent_vectors))
print(len(sent_vectors[0]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


364171
50


In [ ]:
#Tf-Idf weighted word2vec

In [ ]:
tfidf_feat = tf_idf_vect.get_feature_names()
tfidf_sent_vectors = []
row = 0

for sent in list_of_sent:
  sent_vec = np.zeros(50)
  weighted_sum = 0
  for word in sent:
    try:
      vec = w2v_model.wv[word]
      tfidf = final_tf_idf[row, tfidf_feat.index(word)]
      sent_vec += (vec*tf_idf)
      weighted_sum +=tf_idf
    except:
      pass
  sent_vec /= weighted_sum
  tfidf_sent_vectors.append(sent_vec)
  row+=1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()
